# Coursera Capstone Final Project (Week 1-2)

# By Ramón Galván Calderón

## Introduction

Mexico City is the capital and the largest is the capital and largest city of Mexico and the most populous city in North America.

Mexico City is one of the most important cultural and financial centres in the world, it is worth noting that the restaurant industry is one of the main economic sectors in Mexico.

According to the National Institute of Statistics and Geography (INEGI), there are currently around 451,854 food and beverage preparation establishments in Mexico, and it is estimated that they have more than 1.5 million employed people.

The following list shows some of the types of establishments that exist in the Mexican market:

-Seafood Restaurants

-Cafeterias

-Fast Food Restaurants

-Self-service restaurants 

## Problem 

This project will be of interest to those interested in opening a restaurant in Mexico City or analizing data related to food and beverage preparation establishments and their location in Mexico City.

In this project, all the restaurants (available in Foursquare) in Mexico City will be plotted in a map, they will be clustered and classify according to the food type they specialize in. This will help us to determine:

-The areas where there are more restaurants.

-Presence of possible competitors in certain areas of the city.

-The most suitable area to open a restaurant based on the presence of other restaurants.

## Data

I used Forsquare API to get Food Establishments in Mexico City

The queries called during this project were: 

-Food Courts

-Fast Food Restaurants

-Restaurants

-Cafeterias

-Coffee Shops


## Methodology 

My data which has main components Name of the Stablishment, Category, Latitude and Longitude informations of the city.

I used python folium library to visualize the geographical location of Food Establishments in Mexico City and I created a map of the City with their locations superimposed on top. I used latitude and longitude values to get the visual as will be seen below.

I utilized the Foursquare API to explore the food stablishments. I designed the limit as 1000 from their given latitude and longitude informations.

## Code

In [1]:
import numpy as np 

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium 

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1

In [4]:
#Foursquare Credentials
CLIENT_ID = 'ZDTIEWTR4XS1LKOGXE2HKJQJMHIFQVOC1YYPMNGWOFHAUT5I' 
CLIENT_SECRET = 'R3YKTUCW5RFNMMENLJYSDWXE43QKC0VKGTAKI0GSFQ2MPESM' 
VERSION = '20180605' 

#Geographical Coordinates of Mexico City 
MC_Latitude = 19.4284706
MC_Longitude = -99.1276627

In [33]:
# Function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [34]:
#Restaurants
# limit of number of venues returned by Foursquare API
LIMIT = 1000 

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&query=food&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    MC_Latitude, 
    MC_Longitude,
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=ZDTIEWTR4XS1LKOGXE2HKJQJMHIFQVOC1YYPMNGWOFHAUT5I&client_secret=R3YKTUCW5RFNMMENLJYSDWXE43QKC0VKGTAKI0GSFQ2MPESM&v=20180605&ll=19.4284706,-99.1276627&query=food&query=fastfood&limit=1000'

In [35]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5eb962a6618f43001b440294'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'suggestedRadius': 854,
  'headerLocation': 'Centro',
  'headerFullLocation': 'Centro, Mexico City',
  'headerLocationGranularity': 'neighborhood',
  'query': 'food',
  'totalResults': 76,
  'suggestedBounds': {'ne': {'lat': 19.43606398499584,
    'lng': -99.12027177644208},
   'sw': {'lat': 19.4209668652409, 'lng': -99.13597998249003}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd5ea83cfa7b713b2ac26da',
       'name': 'Al Andalus',
       'location': {'address': 'Mesones 171',
        'crossStreet': 'Entre Las Cruces y Jesús María',
        'lat': 19.427880891457818,
        'lng':

In [42]:
venues = results['response']['groups'][0]['items']
    
food = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
food = food.loc[:, filtered_columns]

# filter the category for each row
food['venue.categories'] = food.apply(get_category_type, axis=1)

# clean columns
food.columns = [col.split(".")[-1] for col in food.columns]

print(food.shape)
food.head()

(76, 4)


,name,categories,lat,lng
0,Al Andalus,Middle Eastern Restaurant,19.427881,-99.129224
1,El Antiguo Edhen,Falafel Restaurant,19.430340,-99.129250
2,"Barbacoa ""El Genrry""",Taco Place,19.426894,-99.126917
3,Ehden,Middle Eastern Restaurant,19.430328,-99.129244
4,Chilli-Aquilli,Restaurant,19.428940,-99.126986


In [44]:
#Cafeteria
# limit of number of venues returned by Foursquare API
LIMIT = 1000 

url2 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&query=cafeteria&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    MC_Latitude, 
    MC_Longitude,
    LIMIT)
url2 # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=ZDTIEWTR4XS1LKOGXE2HKJQJMHIFQVOC1YYPMNGWOFHAUT5I&client_secret=R3YKTUCW5RFNMMENLJYSDWXE43QKC0VKGTAKI0GSFQ2MPESM&v=20180605&ll=19.4284706,-99.1276627&query=cafeteria&limit=1000'

In [48]:
results2 = requests.get(url2).json()
results2

{'meta': {'code': 200, 'requestId': '5eb9624e02a172001b16120a'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'suggestedRadius': 854,
  'headerLocation': 'Centro',
  'headerFullLocation': 'Centro, Mexico City',
  'headerLocationGranularity': 'neighborhood',
  'query': 'cafeteria',
  'totalResults': 63,
  'suggestedBounds': {'ne': {'lat': 19.444619541412365,
    'lng': -99.1138302706327},
   'sw': {'lat': 19.41235969805831, 'lng': -99.14711467447701}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '526be56e498e6fb360d8c50d',
       'name': 'la cumbancha',
       'location': {'address': 'Isabel la catolica',
        'crossStreet': '5 DE MAYO',
        'lat': 19.434475064738336,
        'lng': -99.1

In [49]:
venues2 = results2['response']['groups'][0]['items']
    
cafe = json_normalize(venues2) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
cafe = cafe.loc[:, filtered_columns]

# filter the category for each row
cafe['venue.categories'] = cafe.apply(get_category_type, axis=1)

# clean columns
cafe.columns = [col.split(".")[-1] for col in cafe.columns]

print(cafe.shape)
cafe.head()

(35, 4)


,name,categories,lat,lng
0,la cumbancha,Cafeteria,19.434475,-99.136126
1,Maison Kayser,Coffee Shop,19.433198,-99.134317
2,Pastelería Madrid,Bakery,19.429716,-99.134742
3,Da Silva Panadería Artesanal,Coffee Shop,19.434551,-99.138196
4,Vips,Cafeteria,19.419543,-99.129956


In [56]:
#Fast Food
# limit of number of venues returned by Foursquare API
LIMIT = 1000 

url3 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&query=fast_food&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    MC_Latitude, 
    MC_Longitude,
    LIMIT)
url3 # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=ZDTIEWTR4XS1LKOGXE2HKJQJMHIFQVOC1YYPMNGWOFHAUT5I&client_secret=R3YKTUCW5RFNMMENLJYSDWXE43QKC0VKGTAKI0GSFQ2MPESM&v=20180605&ll=19.4284706,-99.1276627&query=fast_food&limit=1000'

In [57]:
results3 = requests.get(url3).json()
results3

{'meta': {'code': 200, 'requestId': '5eb965d39c6f59001cb69b22'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'suggestedRadius': 2562,
  'headerLocation': 'Centro',
  'headerFullLocation': 'Centro, Mexico City',
  'headerLocationGranularity': 'neighborhood',
  'query': 'fast_food',
  'totalResults': 58,
  'suggestedBounds': {'ne': {'lat': 19.4747068415242,
    'lng': -99.07325354717355},
   'sw': {'lat': 19.385945330452113, 'lng': -99.18824739919128}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b89a1d7f964a520bf4732e3',
       'name': "Fisher's Centro",
       'location': {'address': '5 de febrero 24',
        'crossStreet': 'Republica de Uruguay',
        'lat': 19.43016437780471,
        '

In [58]:
venues3 = results3['response']['groups'][0]['items']
    
ff = json_normalize(venues3) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
ff = ff.loc[:, filtered_columns]

# filter the category for each row
ff['venue.categories'] = ff.apply(get_category_type, axis=1)

# clean columns
ff.columns = [col.split(".")[-1] for col in ff.columns]

print(ff.shape)
ff.head()

(33, 4)


,name,categories,lat,lng
0,Fisher's Centro,Seafood Restaurant,19.430164,-99.134566
1,El hijo de Don Toribio,Bistro,19.433456,-99.145200
2,Plaza Alameda,Shopping Mall,19.435031,-99.146687
3,ViveLaPizza,Pizza Place,19.435840,-99.138018
4,Sushi Hoko-Ki,Sushi Restaurant,19.438078,-99.151252


In [71]:
#Shopping Mall
# limit of number of venues returned by Foursquare API
LIMIT = 1000 

url4 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&query=food_court&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    MC_Latitude, 
    MC_Longitude,
    LIMIT)
url4 # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=ZDTIEWTR4XS1LKOGXE2HKJQJMHIFQVOC1YYPMNGWOFHAUT5I&client_secret=R3YKTUCW5RFNMMENLJYSDWXE43QKC0VKGTAKI0GSFQ2MPESM&v=20180605&ll=19.4284706,-99.1276627&query=food_court&limit=1000'

In [72]:
results4 = requests.get(url4).json()
results4

{'meta': {'code': 200, 'requestId': '5eb977cd006dce001c58067f'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'suggestedRadius': 2562,
  'headerLocation': 'Centro',
  'headerFullLocation': 'Centro, Mexico City',
  'headerLocationGranularity': 'neighborhood',
  'query': 'food_court',
  'totalResults': 38,
  'suggestedBounds': {'ne': {'lat': 19.62700809265756,
    'lng': -98.14733879314606},
   'sw': {'lat': 19.05120481498878, 'lng': -99.32600252218312}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b1480adf964a520f4a323e3',
       'name': 'Forum Buenavista',
       'location': {'address': 'Eje 1 Nte. (Mosqueta) 259',
        'crossStreet': 'Av. de los Insurgentes Nte.',
        'lat': 19.448891

In [73]:
venues4 = results4['response']['groups'][0]['items']
    
fc = json_normalize(venues4) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
fc = fc.loc[:, filtered_columns]

# filter the category for each row
fc['venue.categories'] = fc.apply(get_category_type, axis=1)

# clean columns
fc.columns = [col.split(".")[-1] for col in fc.columns]

print(ff.shape)
fc.head()

(33, 4)


,name,categories,lat,lng
0,Forum Buenavista,Shopping Mall,19.448892,-99.151888
1,Mercado Roma,Food Court,19.414187,-99.164575
2,Experiencia Gourmet,Gastropub,19.434499,-99.183020
3,Terminal 2,Airport Terminal,19.421606,-99.078160
4,Food Court,Food Court,19.411250,-99.154991


In [131]:
#Coffee
# limit of number of venues returned by Foursquare API
LIMIT = 1000 

url5 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&query=coffee&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    MC_Latitude, 
    MC_Longitude,
    LIMIT)
url5 # display URL

results5 = requests.get(url5).json()
results5

venues5 = results5['response']['groups'][0]['items']
    
cof = json_normalize(venues5) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
cof = cof.loc[:, filtered_columns]

# filter the category for each row
cof['venue.categories'] = cof.apply(get_category_type, axis=1)

# clean columns
cof.columns = [col.split(".")[-1] for col in cof.columns]

print(cof.shape)
cof.head()

(23, 4)


,name,categories,lat,lng
0,"Café ""Equis""",Coffee Shop,19.430218,-99.127033
1,Krispy Kreme,Donut Shop,19.430935,-99.133052
2,Cafe Bagdad,Café,19.427461,-99.127715
3,El Cafe De Tus Ojos,Coffee Shop,19.427140,-99.134091
4,Cielito Querido Cafe,Café,19.432070,-99.134656


In [138]:
Final = pd.concat([food,ff,cafe,fc,cof])
Final = Final.reset_index(drop=True)
Final.drop_duplicates(keep=False,inplace=True)
Final

,name,categories,lat,lng
0,Al Andalus,Middle Eastern Restaurant,19.427881,-99.129224
1,El Antiguo Edhen,Falafel Restaurant,19.430340,-99.129250
2,"Barbacoa ""El Genrry""",Taco Place,19.426894,-99.126917
3,Ehden,Middle Eastern Restaurant,19.430328,-99.129244
4,Chilli-Aquilli,Restaurant,19.428940,-99.126986
5,Tacos Don Chano,Taco Place,19.429156,-99.126608
6,Pan y Circo Pizza,Pizza Place,19.428967,-99.126989
7,Taqueria Los Paisas,Taco Place,19.426869,-99.128850
8,"""Helus"" Productos Árabes",Middle Eastern Restaurant,19.428710,-99.129777
9,Roldan 37,Mexican Restaurant,19.428762,-99.127048


In [134]:
Cat=pd.DataFrame(data= Final['categories'].value_counts().reset_index().values, columns=["Type of Establishment", "Total"])

print('There are', Cat.shape[0], 'different types of food establishments in Mexico City')

Cat

There are 43 different types of food establishments in Mexico City


,Type of Establishment,Total
0,Food Court,26
1,Cafeteria,22
2,Mexican Restaurant,17
3,Taco Place,12
4,Coffee Shop,12
5,Fast Food Restaurant,9
6,Burger Joint,7
7,Restaurant,7
8,College Cafeteria,5
9,Seafood Restaurant,5


## Results

In [139]:
map_MC = folium.Map(location=[MC_Latitude, MC_Longitude], zoom_start=12)

for lat, lng, label in zip(Final['lat'], Final['lng'], Final['categories']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_MC)
    
map_MC

## Conclusion

As a result, the mayor concentrarion of Food Establishments is in Mexico Citys Downtown where thousand of people walk across everyday. For this reason, people interested in setting up a restaurant can either choose the location in Downtown where they might get a mayor number of costumers or a location up to the North of the City where the rents is much lower and there is not a lot of competence.


We must take into account that the data gathered for this project might not reflect the reality in Mexico City due to the fact that thousands of people are part of the informal economy, so their small restaurant, cafeteria or food related business might not be registered in the Foursquare Database.
